# Code for loading music datasets
Using preprocessed data downloaded from: http://www-etud.iro.umontreal.ca/~boulanni/icml2012

In [40]:
import pickle
import numpy as np
import torch

In [41]:
path = './'
dataset_name = 'Piano-midi.de'
dataset_path = path + dataset_name

In [42]:
dataset = pickle.load(open(dataset_path + '.pickle', 'rb'))

In [44]:
train_test = 'train'
print('Dataset keys: ', dataset.keys())
print('Test dataset length: ', len(dataset[train_test]))
print('Example of datapoint = musical piece: ', dataset[train_test][0])
print('Average length of piece musical piece: ', np.mean([len(piece) for piece in dataset[train_test]]))

Dataset keys:  dict_keys(['test', 'train', 'valid'])
Test dataset length:  87
Example of datapoint = musical piece:  [[55, 79], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [60, 84], [58, 82], [60, 84], [62, 86], [63, 87], [65, 89], [62, 86], [63, 87], [62, 86], [60, 84], [60, 84], [58, 82], [58, 82], [56, 80], [56, 80], [43], [50, 55], [55, 59], [59, 62], [62, 67], [67, 71], [71, 74], [59, 67], [62, 71], [67, 74], [71, 79], [74, 83], [79, 86], [74, 83], [71, 79], [67, 74], [62, 71], [59, 67], [67, 74], [62, 71], [59, 67], [55, 62], [55, 59], [50, 55], [43], [55, 79], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [62, 86], [60, 84], [58, 82], [60, 84], [62, 86], [63, 87], [65, 89], [62, 86], [63, 87], [62, 86], [60, 84], [60, 84], [58, 82], [58, 82], [56, 80], [56, 80], [55, 79], [56, 80], [58, 82], [60, 84], [63, 87], [63, 87], [60, 84], [60, 84], [58, 82], [58, 82], [56, 80], [56, 80], [55, 79], [56, 80], [58, 82], [60, 84], [62, 86], [65, 89], 

In [5]:
def chord_to_binary(chord):
    """According to http://www-etud.iro.umontreal.ca/~boulanni/icml2012,
    each chord is a list of the non-zero elements in the piano-roll at this instant.
     (in MIDI note numbers, between 21 and 108 inclusive).
    
    This function transforms the list into a binary vector of length 88 (= 108 - 21 + 1)
    indicating which notes were played in the given chord
    """
    indices = [note - 21 for note in chord]
    binary_vector = torch.zeros(88)
    binary_vector[indices] = 1.0
    return binary_vector

In [6]:
chord = dataset['test'][0][0]
print(chord)
print(chord_to_binary(chord))

[36, 48]
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.])


In [7]:
def piece_to_binary(piece):
    piece_binary = torch.zeros((len(piece), 88))
    for chord_idx in range(len(piece)):
        piece_binary[chord_idx, :] = chord_to_binary(piece[chord_idx])
    return piece_binary

In [8]:
# Input is binarize piece excluding the final chord
piece_binarized = piece_to_binary(dataset['test'][0])
input = piece_binarized[:-1,:]

# Target is binarize piece excluding the first chord
target = piece_binarized[1:,:]

In [9]:
target[0,:]

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.])

In [10]:
def log_loss_chord(chord, predicted_chord):
    return (torch.log(1 - predicted_chord).sum() 
            - torch.log(1 - predicted_chord[chord]).sum() 
            + torch.log(predicted_chord[chord]).sum())

In [11]:
predicted_chord = torch.rand((88))
chord = dataset['test'][0][0]
log_loss_chord(chord, predicted_chord)

tensor(-81.3952)

In [12]:
def log_loss_binary_chord(binary_chord, predicted_chord):
    return (binary_chord * torch.log(predicted_chord) 
            + (1-binary_chord) * (torch.log(1-predicted_chord))).sum()

In [13]:
log_loss_binary_chord(target[0,:]+0.01, target[0,:]+0.01)

tensor(nan)

In [14]:
d = 0.0000001
t_d = target[0,:]/(1+d)+d/2
log_loss_binary_chord(t_d, t_d)

tensor(1.00000e-05 *
       -7.9970)